In [30]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import wrapcauchy

# Función para calcular los ángulos de giro
def calculate_turning_angles(df):
    """Calcula los ángulos de giro entre cada tres puntos consecutivos de una trayectoria."""
    angles = []
    for i in range(2, len(df)):
        # Vectores consecutivos entre tres puntos
        v1 = np.array([df.iloc[i-1]['x_pos'] - df.iloc[i-2]['x_pos'], df.iloc[i-1]['y_pos'] - df.iloc[i-2]['y_pos']])
        v2 = np.array([df.iloc[i]['x_pos'] - df.iloc[i-1]['x_pos'], df.iloc[i]['y_pos'] - df.iloc[i-1]['y_pos']])
        
        # Ángulo de giro
        angle = np.arctan2(v2[1], v2[0]) - np.arctan2(v1[1], v1[0])
        angles.append(np.mod(angle + np.pi, 2 * np.pi) - np.pi)  # Normalizar a (-pi, pi)
    
    return np.array(angles)

# Leer archivos CSV de las trayectorias CRW con diferentes coeficientes de Cauchy
CRW_trajectory_1 = pd.read_csv('crw_6_6.csv')  # Trayectoria con c = 0.6
CRW_trajectory_2 = pd.read_csv('crw_6_9.csv')  # Trayectoria con c = 0.9

# Calcular los ángulos de giro para cada trayectoria
turning_angles_1 = calculate_turning_angles(CRW_trajectory_1)
turning_angles_2 = calculate_turning_angles(CRW_trajectory_2)

# Distribución teórica de Cauchy para comparación
x_vals = np.linspace(-np.pi, np.pi, 1000)
cauchy_dist_1 = wrapcauchy.pdf(x_vals, c=0.6)  # Parámetro c = 0.6
cauchy_dist_2 = wrapcauchy.pdf(x_vals, c=0.9)  # Parámetro c = 0.9

# Visualizar las distribuciones usando Plotly
fig = go.Figure()

# Histograma de los ángulos de giro observados para la primera trayectoria (c = 0.6)
fig.add_trace(go.Histogram(
    x=turning_angles_1,
    nbinsx=500,
    histnorm='probability density',
    name='observed_0.6',
    marker=dict(color='blue', opacity=0.5)
))


# Histograma de los ángulos de giro observados para la segunda trayectoria (c = 0.9)
fig.add_trace(go.Histogram(
    x=turning_angles_2,
    nbinsx=500,
    histnorm='probability density',
    name='observed_0.9',
    marker=dict(color='red', opacity=0.5)
))

# Curva de la distribución de Cauchy teórica para la primera trayectoria (c = 0.6)
fig.add_trace(go.Scatter(
    x=x_vals,
    y=cauchy_dist_1,
    mode='lines',
    name='cauchy_0.6',
    line=dict(color='blue', width=2)
))

# Curva de la distribución de Cauchy teórica para la segunda trayectoria (c = 0.9)
fig.add_trace(go.Scatter(
    x=x_vals,
    y=cauchy_dist_2,
    mode='lines',
    name='cauchy_0.9',
    line=dict(color='red', width=2)
))



# Ajustes finales del gráfico
fig.update_layout(
    title='Turning-angle Distribution: Observed vs Source (Cauchy)',
    xaxis_title='Turning Angle (radians)',
    yaxis_title='Probability Density',
    barmode='overlay',
    showlegend=True
)

# Mostrar gráfico
fig.show()

